#**Описание проекта**
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте `accuracy` на тестовой выборке самостоятельно.

##**Инструкция по выполнению проекта**
1. Откройте файл с данными и изучите его. Путь к файлу: `/datasets/users_behavior.csv`. Скачать датасет
2. Разделите исходные данные на обучающую, валидационную и тестовую выборки.
3. Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.
4. Проверьте качество модели на тестовой выборке.
5. Дополнительное задание: проверьте модели на вменяемость. Ничего страшного, если не получится: эти данные сложнее тех, с которыми вы работали раньше. В следующем курсе подробнее об этом расскажем.

##**Описание данных**
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
 - `сalls` — количество звонков,
 - `minutes` — суммарная длительность звонков в минутах,
 - `messages` — количество sms-сообщений,
 - `mb_used` — израсходованный интернет-трафик в Мб,
 - `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from  sklearn.ensemble import RandomForestClassifier

##**1. Откройте файл с данными и изучите его. Путь к файлу: `/datasets/users_behavior.csv`. Скачать датасет**

In [ ]:
url = 'https://drive.google.com/file/d/1U1Na0ko03wMPE0sK4e73tzI8W9iDo-dj/view?usp=drive_link'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path)
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [ ]:
df.duplicated().sum()

0

In [ ]:
df.isna().sum()

,0
calls,0
minutes,0
messages,0
mb_used,0
is_ultra,0


 - Датасет содержит 3214 строк.

В датасете хранится информация:
  - о количестве звонков,
  - о суммарной продолжительности звонков в минутах,
  - о количестве sms-сообщений,
  - о количестве израсходованного интернет-трафика в Мб,
  - тарифе.

- В датасете нет пропущенных значений и дублирующих строк, поэтому можно работать с ним дальше.

##**2. Разделите исходные данные на обучающую, валидационную и тестовую выборки.**

У нас нет отдельного датасета для тестовой проверки, поэтому разделим наш исходный датасет на три части:
 - обучающую (60%);
 - валидационную (20%);
 - тестовую (20%).
 Разделим наш датасет на признаки (features):
  - `calls`
  - `minutes`
  - `messages`
  - `mb_used`

  Целевая переменная `is_ultra` содержит только значения 0 и 1, поэтому перед нами стоит задача классификации

In [ ]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)

print(f'''Размер обучающей выборки
features: {features_train.shape}   target: {target_train.shape}

Размер валидационной выборки
features: {features_valid.shape}    target: {target_valid.shape}

Размер тестовой выборки
features: {features_test.shape}    target: {target_test.shape}''')

Размер обучающей выборки
features: (1928, 4)   target: (1928,)

Размер валидационной выборки
features: (643, 4)    target: (643,)

Размер тестовой выборки
features: (643, 4)    target: (643,)


##**3. Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.**

###**3.1 Решающее дерево**
Будем менять гиперпараметр - максимальная глубина дерева, и выберем модель с наибольшим показателем `accuracy`.

In [ ]:
for depth in range(1,11):
  model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
  model.fit(features_train, target_train)
  predictions = model.predict(features_valid)
  result = accuracy_score(target_valid, predictions)
  print(f'max_depth = {depth:<2}, accuracy = {result}')


max_depth = 1 , accuracy = 0.7542768273716952
max_depth = 2 , accuracy = 0.7822706065318819
max_depth = 3 , accuracy = 0.7853810264385692
max_depth = 4 , accuracy = 0.7791601866251944
max_depth = 5 , accuracy = 0.7791601866251944
max_depth = 6 , accuracy = 0.7838258164852255
max_depth = 7 , accuracy = 0.7822706065318819
max_depth = 8 , accuracy = 0.7791601866251944
max_depth = 9 , accuracy = 0.7822706065318819
max_depth = 10, accuracy = 0.7744945567651633


Наибольшая метрика accuracy=0.7853 при глубине дерева - 3.

###**3.2 Логистическая регрессия**

In [ ]:
model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
result = accuracy_score(target_valid, predictions)
print(result)

0.7558320373250389


Модель логистическая регрессия показала accuracy = 0.7558

###**3.3 Случайный лес**
Будем менять гиперпараметры n_estimators и max_depth от 1 до 10 и определим лучшую модель по наиболее высокому значению accuracy.

In [ ]:
best_model = None
best_result = 0
est_best = None
depth_best = None
for est in range(1,11):
  for depth in range(1,11):
    model = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions)
    if result > best_result:
      best_result = result
      best_model = model
      est_best = est
      depth_best = depth
print(f'Количество деревьев = {est_best},  max_depth = {depth_best}, accuracy = {best_result}')


Количество деревьев = 8,  max_depth = 8, accuracy = 0.80248833592535


Случайный лес показал значение accuracy=0.8024 при n_estimators=8 и max_depth=8. Это самое высокое значение метрики из всех моделей опробованных в данном проекте.

 - Решающее дерево. Accuracy = 0.7853 при max_depth = 3
 - Логистическая регрессия. Accuracy = 0.7558
 - Случайный лес. Accuracy = 0.8024 при n_estimators = 8 и max_depth = 8.
   

###**4. Проверьте качество модели на тестовой выборке.**
Лучшая модель сохранена в переменную `best_model` - это случайный лес с гиперпараметрами n_estimators = 8 и max_depth = 8. Проверим качество этой модели на тестовой выборке.

In [ ]:
predictions = best_model.predict(features_test)
result = accuracy_score(target_test, predictions)
print(result)

0.7962674961119751


Лучшая модель показала accuracy = 0.7962 на тестовой выборке и 0.8024 на обучающей, что свидетельсвует о том, что гиперпараметры подобраны оптимально и при таких параметрах остсутвует переобучение( разница между точностью на тестовой и обучающей выборке небольшая).

###**5. Дополнительное задание: проверьте модели на вменяемость. Ничего страшного, если не получится: эти данные сложнее тех, с которыми вы работали раньше. В следующем курсе подробнее об этом расскажем.**

Проверка на адекватность будет состоять в сравнении с моделью, которая все время будет предсказывать один и тот же наибольший класс.

In [ ]:
target_test.value_counts()

,count
is_ultra,
0,440
1,203


В нашем случае наибольший класс - 0 (нули).

In [18]:
predictions_zeros = [0]*len(target_test)
result_zeros = accuracy_score(target_test, predictions_zeros)
print(result_zeros)

0.6842923794712286


Случайная модель показала результат 0.6842 на тестовой выборке, а лучшая модель 0.7962. Можно сделать вывод, что лучшая модель работает адекватно.